### Make lists of image and label paths

In [2]:
import os
dir_imgs = "/mnt/sda/deep_learning/CSE527_FinalProject-master/images/train"
dir_labels = "/mnt/sda/deep_learning/CSE527_FinalProject-master/images/label"
assert(os.path.isdir(dir_imgs))
assert(os.path.isdir(dir_labels))

In [3]:
import glob
img_paths = sorted(glob.glob(dir_imgs + "/*.tif"))
label_paths = sorted(glob.glob(dir_labels + "/*.tif"))
assert(len(img_paths) ==len(label_paths))
print(img_paths[1:10])
print(label_paths[1:10])

train_size = (556,556)

['/mnt/sda/deep_learning/CSE527_FinalProject-master/images/train/10.tif', '/mnt/sda/deep_learning/CSE527_FinalProject-master/images/train/100.tif', '/mnt/sda/deep_learning/CSE527_FinalProject-master/images/train/1000.tif', '/mnt/sda/deep_learning/CSE527_FinalProject-master/images/train/1001.tif', '/mnt/sda/deep_learning/CSE527_FinalProject-master/images/train/1002.tif', '/mnt/sda/deep_learning/CSE527_FinalProject-master/images/train/1003.tif', '/mnt/sda/deep_learning/CSE527_FinalProject-master/images/train/1005.tif', '/mnt/sda/deep_learning/CSE527_FinalProject-master/images/train/1006.tif', '/mnt/sda/deep_learning/CSE527_FinalProject-master/images/train/1008.tif']
['/mnt/sda/deep_learning/CSE527_FinalProject-master/images/label/10.tif', '/mnt/sda/deep_learning/CSE527_FinalProject-master/images/label/100.tif', '/mnt/sda/deep_learning/CSE527_FinalProject-master/images/label/1000.tif', '/mnt/sda/deep_learning/CSE527_FinalProject-master/images/label/1001.tif', '/mnt/sda/deep_learning/CSE52

### Utility functions

In [4]:
## load random image and label from list of image and label paths
import random
from PIL import Image
def load_rand_sample(image_paths, label_paths, shape=train_size):
    num_samples = len(image_paths)
    ii_rand = random.randint(0,num_samples-1)
    img_rand = Image.open(image_paths[ii_rand]).resize(shape)
    label_rand = Image.open(label_paths[ii_rand]).resize(shape)
    return img_rand, label_rand

""" Elastic transform code from https://gist.github.com/erniejunior/601cdf56d2b424757de5"""
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter
def elastic_transform_git(image, alpha, sigma, order=0, random_state=None):
    """Elastic deformation of images as described in [Simard2003]_.
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
       Convolutional Neural Networks applied to Visual Document Analysis", in
       Proc. of the International Conference on Document Analysis and
       Recognition, 2003.
    """
    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape
    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha

    x, y, z = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), np.arange(shape[2]))
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1)), np.reshape(z, (-1, 1))

    distored_image = map_coordinates(image, indices, order=order, mode='reflect')
    return distored_image.reshape(image.shape)

""" Elastic transform code from https://gist.github.com/erniejunior/601cdf56d2b424757de5"""
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter
def elastic_transform_git_mod(image, label, alpha, sigma, random_state=None):
    """Elastic deformation of images as described in [Simard2003]_.
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
       Convolutional Neural Networks applied to Visual Document Analysis", in
       Proc. of the International Conference on Document Analysis and
       Recognition, 2003.
    """
    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape
    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha

    x, y, z = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), np.arange(shape[2]))
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1)), np.reshape(z, (-1, 1))
    
    distored_image = map_coordinates(image, indices, order=1, mode='reflect')
    distored_label = map_coordinates(label, indices, order=0, mode='reflect')
    return distored_image.reshape(shape), distored_label.reshape(shape)


import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np

### Test brightness parameter

In [5]:
img, _ = load_rand_sample(img_paths, label_paths)
def adjust_brightness(img, brightness_factor):
    img_adj = TF.adjust_brightness(img, brightness_factor)
    plt.figure(figsize=(25,25))
    plt.subplot(121)
    plt.imshow(img, cmap="gray", vmin=0, vmax=255)
    plt.subplot(122)
    plt.imshow(img_adj, cmap="gray", vmin=0, vmax=255)
    print("sum img:", np.array(img).sum())
    print("sum img_adj:", np.array(img_adj).sum())
interact(adjust_brightness, img=fixed(img), brightness_factor=(0., 2., 0.05))

interactive(children=(FloatSlider(value=1.0, description='brightness_factor', max=2.0, step=0.05), Output()), …

<function __main__.adjust_brightness(img, brightness_factor)>

### Test contrast parameter

In [6]:
img, _ = load_rand_sample(img_paths, label_paths)
def adjust_contrast(img, contrast_factor):
    img_adj = TF.adjust_contrast(img, contrast_factor)
    plt.figure(figsize=(25,25))
    plt.subplot(121)
    plt.imshow(img, cmap="gray", vmin=0, vmax=255)
    plt.subplot(122)
    plt.imshow(img_adj, cmap="gray", vmin=0, vmax=255)
interact(adjust_contrast, img=fixed(img), contrast_factor=(0., 2., 0.05))

interactive(children=(FloatSlider(value=1.0, description='contrast_factor', max=2.0, step=0.05), Output()), _d…

<function __main__.adjust_contrast(img, contrast_factor)>

### Test affine transformation parameters

In [13]:
img, label = load_rand_sample(img_paths, label_paths)
def affine_transform(img, label, angle, translate_x, translate_y, scale, shear):
    img_trf = TF.affine(img, angle, (translate_x, translate_y), scale, shear, resample=0, fillcolor=None)
    label_trf = TF.affine(label, angle, (translate_x, translate_y), scale, shear, resample=0, fillcolor=None)
    plt.figure(figsize=(25,25))
    plt.subplot(121)
    plt.imshow(img_trf, cmap="gray", vmin=0, vmax=255)
    plt.subplot(122)
    plt.imshow(label_trf, cmap="gray")
interact(affine_transform, img=fixed(img), label=fixed(label), angle=(-30,30,1), translate_x=(-100,100,1), translate_y=(-100,100,1), scale=(0.9,1.1,0.05), shear=(-6,6,0.05))

interactive(children=(IntSlider(value=0, description='angle', max=30, min=-30), IntSlider(value=0, description…

<function __main__.affine_transform(img, label, angle, translate_x, translate_y, scale, shear)>

In [8]:
angle=30
translate = (50, 50)
scale = 1.5
shear = 6

In [14]:
%%timeit
img_trf = TF.affine(img, angle, translate, scale, shear, resample=0, fillcolor=None)
label_trf = TF.affine(label, angle, translate, scale, shear, resample=0, fillcolor=None)

613 µs ± 294 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Test elastic transform parameters

In [31]:
img, label = load_rand_sample(img_paths, label_paths)
def elastic_transform(img, label, alpha, sigma):
    img_trf = elastic_transform_git(np.array(img)[:,:,np.newaxis], alpha, sigma)[:,:,0]
    #label_trf = elastic_transform_git(label, alpha, sigma)
    plt.figure(figsize=(25,25))
    plt.subplot(121)
    plt.imshow(img, cmap="gray", vmin=0, vmax=255)
    plt.subplot(122)
    plt.imshow(img_trf, cmap="gray", vmin=0, vmax=255)
interact(elastic_transform, img=fixed(img), label=fixed(label), alpha=(0,1000,1), sigma=(4,7,0.01))

interactive(children=(IntSlider(value=500, description='alpha', max=1000), FloatSlider(value=5.0, description=…

<function __main__.elastic_transform(img, label, alpha, sigma)>

In [16]:
img, label = load_rand_sample(img_paths, label_paths)
def elastic_transform(img, label, alpha, sigma):
    img_trf_0 = elastic_transform_git(np.array(img)[:,:,np.newaxis], alpha, sigma)[:,:,0]
    img_trf_1 = elastic_transform_git(np.array(img)[:,:,np.newaxis], alpha, sigma, order=1)[:,:,0]
    #label_trf = elastic_transform_git(label, alpha, sigma)
    plt.figure(figsize=(35,35))
    plt.subplot(131)
    plt.imshow(img_trf_0, cmap="gray", vmin=0, vmax=255)
    plt.subplot(132)
    plt.imshow(img, cmap="gray", vmin=0, vmax=255)
    plt.subplot(133)
    plt.imshow(img_trf_1, cmap="gray", vmin=0, vmax=255)
interact(elastic_transform, img=fixed(img), label=fixed(label), alpha=(0,1400,1), sigma=(5,6.5,0.01))

interactive(children=(IntSlider(value=700, description='alpha', max=1400), FloatSlider(value=5.75, description…

<function __main__.elastic_transform(img, label, alpha, sigma)>

In [84]:
checkerboard = np.zeros((1133, 1133), dtype = int) 
# fill with 1 the alternate rows and columns 
checkerboard[1::2, ::5] = 1
checkerboard[::5, 1::2] = 1

In [88]:
checkerboard = Image.open("checkerboard.png")

In [97]:
checkerboard = np.array(checkerboard)[:, 3:-1]

In [60]:
%%timeit
alpha = 700
sigma = 5.75
img_trf_0 = elastic_transform_git(np.array(img)[:,:,np.newaxis], alpha, sigma)[:,:,0]

53.8 ms ± 168 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [59]:
%%timeit
alpha = 700
sigma = 5.75
img_trf_0 = elastic_transform_git(np.array(img)[:,:,np.newaxis], alpha, sigma, order=1)[:,:,0]

68.3 ms ± 334 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [58]:
%%timeit
comb_img = np.concatenate((np.array(img)[..., None], np.array(label)[..., None]), axis=2)
img_trf_0 = elastic_transform_git(comb_img, alpha, sigma, order=1)[:,:,0]

119 ms ± 348 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [57]:
%%timeit
comb_img = np.concatenate((np.array(img)[..., None], np.array(label)[..., None]), axis=2)
img_trf_0 = elastic_transform_git(comb_img, alpha, sigma, order=0)
img_1 = comb_img[..., 0]
label_1 = comb_img[..., 1]

96.2 ms ± 167 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [118]:
%%timeit
img_2, label_2 = elastic_transform_git_mod(np.array(img)[..., None], np.array(label)[..., None], alpha, sigma)

76.2 ms ± 67.1 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [38]:
comb_img = np.concatenate((np.array(img)[..., None], np.array(label)[..., None]), axis=2)
comb_img.shape


(556, 556, 2)

In [119]:
img, label = load_rand_sample(img_paths, label_paths)
def elastic_transform(img, label, alpha, sigma):
    img_trf, label_trf = elastic_transform_git_mod(np.array(img)[..., None], np.array(label)[..., None], alpha, sigma)
    label_trf_rgb = np.concatenate((label_trf==0,label_trf==1,label_trf==2), 2).astype("float")
    label = np.array(label)[...,None]
    label_rgb = np.concatenate((label==0,label==1,label==2), 2).astype("float")
    plt.figure(figsize=(30,15))
    plt.subplot(121)
    plt.imshow(img_trf[...,0], cmap="gray")
    plt.imshow(label_trf_rgb, cmap="Greens", alpha=0.15)
    plt.subplot(122)
    plt.imshow(np.array(img), cmap="gray")
    plt.imshow(label_rgb, cmap="Greens", alpha=0.15)
    
    
interact(elastic_transform, img=fixed(img), label=fixed(label), alpha=(0,1400,1), sigma=(5,6.5,0.01))


interactive(children=(IntSlider(value=700, description='alpha', max=1400), FloatSlider(value=5.75, description…

<function __main__.elastic_transform(img, label, alpha, sigma)>